In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

In [ ]:
# Reads chess move image which contains multiple symbols
chmove_image = cv2.imread('images\\sheets\\notex3_b1.png')

# Converts RGB to Gray scale
chmove_image_gray = cv2.cvtColor(chmove_image, cv2.COLOR_BGR2GRAY)

# Uses Otsu method for thresholding the image 
ret, chmove_image_thresh = cv2.threshold(chmove_image_gray, 0, 255, cv2.THRESH_OTSU)

ctrs, _ = cv2.findContours(chmove_image_thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

# Calculate average bounding contour area
areas = [cv2.contourArea(ctr) for ctr in ctrs]
average_area = sum(areas) / len(areas)

# Define thresholds as a percentage of the average area
lower_threshold = 625  # 0.5 * average_area
upper_threshold = 6400 # 6.0 * average_area

extracted_symbols = []
for i, ctr in enumerate(sorted_ctrs):
    x, y, w, h = cv2.boundingRect(ctr)
    area = w*h
    if lower_threshold < area < upper_threshold:
        # rect = cv2.rectangle(chmove_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # cv2.imshow('rect', rect)
        extracted_symbols.append(chmove_image_thresh[y:y + h, x:x + w])

# cv2.imshow('chmove_image', chmove_image)
# cv2.imshow('chmove_image_thresh', chmove_image_thresh)
# cv2.waitKey(0)


In [ ]:
# Load model
model = tf.keras.models.load_model("models\channo_v0.4.keras")

# Define result list
result = []

# Testing
filtered_images = []

for extracted_symbol in extracted_symbols:
    # Define the desired size of the square image (AxA)
    desired_size = max(extracted_symbol.shape) + 10  # offset

    # Create a blank square image of the desired size
    resized_image = np.ones((desired_size, desired_size), dtype=np.uint8) * 255

    # Calculate the position to place the original image in the center
    x_offset = (desired_size - extracted_symbol.shape[1]) // 2
    y_offset = (desired_size - extracted_symbol.shape[0]) // 2

    # Place the original image in the center of the blank square image
    resized_image[y_offset:y_offset + extracted_symbol.shape[0], x_offset:x_offset + extracted_symbol.shape[1]] = extracted_symbol

    # Downscale the resized image to the target size (e.g., 28x28)
    downscaled_image = cv2.resize(resized_image, (28, 28))

    # Invert, normalize and reshape image to give input our model
    filtered_image = 255 - downscaled_image             # Invert

    # Testing
    # filtered_images.append(cv2.GaussianBlur(filtered_image, (5, 5), 0))
    filtered_images.append(filtered_image)
    
    filtered_image = filtered_image / 255.0             # Normalize
    filtered_image = filtered_image.reshape(1, 28, 28)  # Reshape

    # Pass filtered image to our model
    predictions = model.predict(filtered_image)
    predicted_class = np.argmax(predictions[0])
    print(predictions)
    result.append(predicted_class)

# Plotting all the images
fig, axes = plt.subplots(nrows=1, ncols=len(filtered_images), figsize=(7, 7))
for ax, img in zip(axes, filtered_images):
    ax.imshow(img, cmap='gray')
    ax.axis('off')
plt.show()

print(result)

In [ ]:
def predict_symbols(chmove_image):
    # Converts RGB to Gray scale
    chmove_image_gray = cv2.cvtColor(chmove_image, cv2.COLOR_BGR2GRAY)

    # Uses Otsu method for thresholding the image 
    ret, chmove_image_thresh = cv2.threshold(chmove_image_gray, 0, 255, cv2.THRESH_OTSU)

    ctrs, _ = cv2.findContours(chmove_image_thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

    # Calculate average bounding contour area
    areas = [cv2.contourArea(ctr) for ctr in ctrs]
    average_area = sum(areas) / len(areas)

    # Define thresholds as a percentage of the average area
    lower_threshold = 625  # 0.5 * average_area
    upper_threshold = 6400 # 6.0 * average_area

    extracted_symbols = []
    for i, ctr in enumerate(sorted_ctrs):
        x, y, w, h = cv2.boundingRect(ctr)
        area = w*h
        if lower_threshold < area < upper_threshold:
            extracted_symbols.append(chmove_image_thresh[y:y + h, x:x + w])

    # Load model
    model = tf.keras.models.load_model("models\channo_v0.4.keras")

    # Define result list
    result = []

    for extracted_symbol in extracted_symbols:
        # Define the desired size of the square image (AxA)
        desired_size = max(extracted_symbol.shape) + 10  # offset

        # Create a blank square image of the desired size
        resized_image = np.ones((desired_size, desired_size), dtype=np.uint8) * 255

        # Calculate the position to place the original image in the center
        x_offset = (desired_size - extracted_symbol.shape[1]) // 2
        y_offset = (desired_size - extracted_symbol.shape[0]) // 2

        # Place the original image in the center of the blank square image
        resized_image[y_offset:y_offset + extracted_symbol.shape[0], x_offset:x_offset + extracted_symbol.shape[1]] = extracted_symbol

        # Downscale the resized image to the target size (e.g., 28x28)
        downscaled_image = cv2.resize(resized_image, (28, 28))

        # Invert, normalize and reshape image to give input our model
        filtered_image = 255 - downscaled_image             # Invert
        filtered_image = filtered_image / 255.0             # Normalize
        filtered_image = filtered_image.reshape(1, 28, 28)  # Reshape

        # Pass filtered image to our model
        predictions = model.predict(filtered_image)
        predicted_class = np.argmax(predictions[0])
        result.append(predicted_class)

    return result

In [ ]:
chmove_images = []
chmove_expection_list = []
chmove_prediction_list = []

for i in range(5):
    chmove_image_w = cv2.imread(f'images\\sheets\\notex3_w{(i + 1)}.png')
    chmove_image_b = cv2.imread(f'images\\sheets\\notex3_b{(i + 1)}.png')
    chmove_image_w[chmove_image_w > 200] = 180
    chmove_image_b[chmove_image_b > 200] = 180
    chmove_images.append(chmove_image_w)
    chmove_images.append(chmove_image_b)

chmove_expection_list.append([13, 4]);          chmove_expection_list.append([13, 5])
chmove_expection_list.append([21, 14, 3]);      chmove_expection_list.append([21, 11, 6])
chmove_expection_list.append([20, 10, 5]);      chmove_expection_list.append([9, 6])
chmove_expection_list.append([20, 9, 4]);       chmove_expection_list.append([21, 14, 6])
chmove_expection_list.append([12, 3]);          chmove_expection_list.append([20, 13, 7])
# chmove_expection_list.append([20, 23, 11, 6]);  chmove_expection_list.append([10, 23, 11, 6])
# chmove_expection_list.append([21, 23, 13, 5]);  chmove_expection_list.append([20, 10, 7])
# chmove_expection_list.append([0, 22, 0]);       chmove_expection_list.append([16, 6])
# chmove_expection_list.append([21, 11, 3]);      chmove_expection_list.append([0, 22, 0])
# chmove_expection_list.append([20, 13, 3]);      chmove_expection_list.append([12, 5])

for chmove_image in chmove_images:
    prediction = predict_symbols(chmove_image)
    chmove_prediction_list.append(prediction)

total_sample = len(chmove_expection_list)
total_correct_prediction = 0
for i in range(len(chmove_expection_list)):
    if len(chmove_expection_list[i]) == len(chmove_prediction_list[i]):
        if chmove_expection_list[i] == chmove_prediction_list[i]:
            total_correct_prediction += 1

print(f'Accuracy: {(total_correct_prediction / total_sample)}')
